In [1]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
from xfem import *
from xfem.lsetcurv import *
from math import pi,e
from numpy import linspace

ngsglobals.msg_level = 2
DrawDC = MakeDiscontinuousDraw(Draw)

interactive = 1

importing ngsxfem-2.1.2504


In [2]:
# physical parameters for Biot
mu  = 10
lam = 100
alpha = 1
# alpha = 0
K = 10 # k^-1
nu = 1
# s0 = 10
s0 = 1e-2

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(2)

In [3]:
# DG space order
order_eta = 2
order_u = 2
order_p = 1
# penalty parameters
beta_eta = 100
beta_u = 100
# ghost penalty parameters
gamma_s = 0.1
gamma_u = 0.1
gamma_p = 0.1
gamma_m = 0
tau_p = 0

In [4]:
# Manufactured exact solution for monitoring the error
eta_x = sin(pi*x)*sin(pi*y)
eta_y = x*y*(x-1)*(y-1)
u_x = sin(pi*x) * cos(pi*y)
u_y = -cos(pi*x) * sin(pi*y)
exact_p = sin(pi*(x-y))


#---------------------Example 3 -----------------------
# u_x = sin(pi*x) * cos(pi*y)
# u_y = -cos(pi*x) * sin(pi*y)
# eta_x = sin(pi*x)*sin(pi*y) 
# eta_y = cos(pi*x)*cos(pi*y) 
# exact_p = sin(pi*(x-y))

exact_eta = CF((eta_x, eta_y))
exact_u = CF((u_x,u_y))

# strain tensor
epsilon_xx = eta_x.Diff(x)
epsilon_yy = eta_y.Diff(y) 
epsilon_xy = 0.5*(eta_x.Diff(y) +  eta_y.Diff(x))

# total stress tensor
sigma_xx = lam*(epsilon_xx + epsilon_yy) + 2*mu*epsilon_xx - alpha*exact_p
sigma_yy = lam*(epsilon_xx + epsilon_yy) + 2*mu*epsilon_yy - alpha*exact_p
sigma_xy = 2*mu*epsilon_xy

# 右端项 f_x, f_y
f_x = - (sigma_xx.Diff(x) + sigma_xy.Diff(y))
f_y = - (sigma_xy.Diff(x) + sigma_yy.Diff(y))

fm_x = K*u_x-nu*(u_x.Diff(x).Diff(x)+u_x.Diff(y).Diff(y)) + exact_p.Diff(x)
fm_y = K*u_y-nu*(u_y.Diff(x).Diff(x)+u_y.Diff(y).Diff(y)) + exact_p.Diff(y)

# 向量形式
fe = CF((f_x, f_y))
fm = CF((fm_x, fm_y))
fp = s0*exact_p+alpha*(eta_x.Diff(x)+eta_y.Diff(y)) + u_x.Diff(x) + u_y.Diff(y)

etaD = exact_eta
uD = exact_u
pD = exact_p

In [5]:
# Construct background mesh
# Quadrilateral (or simplicial mesh)
quad_mesh = False
# Mesh diameter
h0 = 1/10
# Geometry and Mesh
square = SplineGeometry()
square.AddRectangle((-1, -0.5), (1, 0.5), bc=1)
ngmesh = square.GenerateMesh(maxh=h0, quad_dominated=quad_mesh)
mesh = Mesh(ngmesh)
# Draw(mesh)

 Generate Mesh from spline geometry


In [6]:
# multiple level sets
circle1 = 1/9-(x-0.5)**2-y**2
circle2 = 1/9-(x+0.5)**2-y**2
level_sets = (circle1, circle2)
nr_ls = len(level_sets)
lsets_p1 = tuple(GridFunction(H1(mesh,order=1)) for i in range(nr_ls))

In [ ]:
# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsets_p1)
hasneg = ci.GetElementsOfType(HASNEG)
hasif = ci.GetElementsOfType(IF)
# Draw(BitArrayCF(hasneg),mesh)
# Draw(BitArrayCF(hasif),mesh)

# facets used for stabilization:
ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)
ba_surround_facets = GetElementsWithNeighborFacets(mesh,ba_facets)
interior_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasneg)
in_surround_facets = GetElementsWithNeighborFacets(mesh,interior_facets)
# Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets") 
# Draw(BitArrayCF(in_surround_facets), mesh, "surrounding_facets") 


In [ ]:
kappaminus = CutRatioGF(ci)
kappaminus_values = kappaminus.vec.FV().NumPy()

# 找出非零值中的最小值
# 这里需要过滤掉那些没有被切割的单元的值（通常是0）
# min_value = 1.0 # 初始化一个比任何比率都大的值
# for value in kappaminus_values:
#     if value > 0 and value < min_value:
#         min_value = value

# print(f"所有被切割单元中，比值的最小值为: {min_value}")

# Or
positive_values = [v for v in kappaminus_values if v > 0]
if positive_values:
    min_value_pythonic = min(positive_values)
    print(f"所有被切割单元中，比值的最小值为 (Pythonic): {min_value_pythonic}")
else:
    print("没有找到被切割的单元。")

In [ ]:
#Unfitted DG spaces
Ehbase = VectorL2(mesh, order=order_eta, dirichlet=[], dgjumps=True) # space for displacement
Uhbase = VectorL2(mesh, order=order_u, dirichlet=[], dgjumps=True) # space for velocity
# Phbase = L2(mesh, order=order_p, dirichlet=[], dgjumps=True) # space for pressure 
Phbase = H1(mesh, order=order_p, dirichlet=[], dgjumps=True) # space for pressure 
E = Compress(Ehbase, GetDofsOfElements(Ehbase, ci.GetElementsOfType(HASNEG)))
U = Compress(Uhbase, GetDofsOfElements(Uhbase, ci.GetElementsOfType(HASNEG)))
P = Compress(Phbase, GetDofsOfElements(Phbase, ci.GetElementsOfType(HASNEG)))
fes = E*U*P
(eta,u,p), (kxi,v,q) = fes.TnT()

In [ ]:
# Define the jumps and the averages
# n = Normalize(grad(lsetp1)) # outer normal vector on the boundary
n = 1.0 / Norm(grad(lsetp1)) * grad(lsetp1)
ne = specialcf.normal(2) # normal vectors on faces
h = specialcf.mesh_size  

strain_eta = Sym(Grad(eta))
strain_kxi = Sym(Grad(kxi))
mean_stress_eta = 0.5*(Stress(Sym(Grad(eta)))+Stress(Sym(Grad(eta.Other()))))*ne
mean_stress_kxi = 0.5*(Stress(Sym(Grad(kxi)))+Stress(Sym(Grad(kxi.Other()))))*ne
jump_eta = eta - eta.Other()
jump_kxi = kxi - kxi.Other()


jump_p = p - p.Other()
jump_q = q - q.Other()
mean_p = 0.5*(p + p.Other())
mean_q = 0.5*(q + q.Other())


mean_dudn = 0.5*(Grad(u)+Grad(u.Other()))*ne
mean_dvdn = 0.5*(Grad(v)+Grad(v.Other()))*ne
jump_u = u - u.Other()
jump_v = v - v.Other()
 

In [ ]:
# integration domains:
domega = dCut(lsetp1, NEG, definedonelements=hasneg, deformation=deformation)
dk = dCut(lsetp1, NEG, skeleton=True, definedonelements=interior_facets,
          deformation=deformation)
ds = dCut(lsetp1, IF, definedonelements=hasif, deformation=deformation)
dw = dFacetPatch(definedonelements=ba_facets, deformation=deformation)
# df = dx(skeleton=True,definedonelements=interior_facets,deformation=deformation)
 # = dFacetPatch(definedonelements=interior_facets, deformation=deformation)

In [ ]:
Ah = BilinearForm(fes)
# Ae
Ah += 2*mu*InnerProduct(strain_eta,strain_kxi)*domega + lam*div(eta)*div(kxi)*domega \
        - (InnerProduct(mean_stress_eta,jump_kxi) + InnerProduct(mean_stress_kxi,jump_eta) - beta_eta/h*InnerProduct(jump_eta,jump_kxi))*dk \
        - (InnerProduct(Stress(Sym(Grad(eta)))*n,kxi) + InnerProduct(Stress(Sym(Grad(kxi)))*n,eta) - beta_eta/h*InnerProduct(eta,kxi))*ds
# order=1 i_s 
# Ah += gamma_s * h * InnerProduct(Grad(eta)*ne - Grad(eta.Other())*ne,Grad(kxi)*ne - Grad(kxi.Other())*ne) * dw
Ah += gamma_s / (h**2) * (eta - eta.Other()) * (kxi - kxi.Other()) * dw

# Be
Ah += -alpha*(div(kxi)*p*domega - mean_p*jump_kxi*ne*dk - p*kxi*n*ds)

              
# Am
Ah += nu*InnerProduct(Grad(u),Grad(v))*domega \
        - nu*(InnerProduct(mean_dudn,jump_v) + InnerProduct(mean_dvdn,jump_u) - beta_u/h*InnerProduct(jump_u,jump_v))*dk \
        - nu*(InnerProduct(Grad(u)*n,v) + InnerProduct(Grad(v)*n,u) - beta_u/h*InnerProduct(u,v))*ds\
        + K*InnerProduct(u,v)*domega
# ghost penalty for velocity
# Ah += gamma_u * h * InnerProduct(Grad(u)*ne - Grad(u.Other())*ne,Grad(v)*ne - Grad(v.Other())*ne) * dw
Ah += gamma_u / (h**2) * (u - u.Other()) * (v - v.Other()) * dw



 # Bm 
Ah += -div(v)*p*domega + mean_p*jump_v*ne*dk + p*v*n*ds

# -Bm
Ah += div(u)*q*domega - mean_q*jump_u*ne*dk - q*u*n*ds

# order=1 i_p 
Ah += gamma_p * (h**3) * (grad(p)*ne - grad(p.Other())*ne)*(grad(q)*ne - grad(q.Other())*ne) * dk
# Ah += gamma_p * jump_p * jump_q * df

# M
Ah += s0*p*q*domega + gamma_m*(h**3)*(grad(p)*ne - grad(p.Other())*ne)*(grad(q)*ne - grad(q.Other())*ne) * dw

# -Be
Ah += alpha*(div(eta)*q*domega - mean_q*jump_eta*ne*dk - q*eta*n*ds)

# stabilization
Ah += tau_p*h*jump_p*jump_q*dk
Ah.Assemble()

In [ ]:
# Ah = BilinearForm(fes)
# # Ae
# Ah += 2*mu*InnerProduct(strain_eta,strain_kxi)*domega + lam*div(eta)*div(kxi)*domega \
#         - (InnerProduct(mean_stress_eta,jump_kxi) + InnerProduct(mean_stress_kxi,jump_eta) - beta_eta/h*InnerProduct(jump_eta,jump_kxi))*dx(skeleton=True) \
#         - (InnerProduct(Stress(Sym(Grad(eta)))*n,kxi) + InnerProduct(Stress(Sym(Grad(kxi)))*n,eta) - beta_eta/h*InnerProduct(eta,kxi))*ds
# # order=1 i_s 
# Ah += gamma_s * h * InnerProduct(Grad(eta)*ne - Grad(eta.Other())*ne,Grad(kxi)*ne - Grad(kxi.Other())*ne) * dw

# # Be
# Ah += -alpha*(div(kxi)*p*domega - mean_p*jump_kxi*ne*dx(skeleton=True) - p*kxi*n*ds)

              
# # Am
# Ah += nu*InnerProduct(Grad(u),Grad(v))*domega \
#         - nu*(InnerProduct(mean_dudn,jump_v) + InnerProduct(mean_dvdn,jump_u) - beta_u/h*InnerProduct(jump_u,jump_v))*dx(skeleton=True) \
#         - nu*(InnerProduct(Grad(u)*n,v) + InnerProduct(Grad(v)*n,u) - beta_u/h*InnerProduct(u,v))*ds\
#         + K*InnerProduct(u,v)*domega

#  # Bm 
# Ah += -div(v)*p*domega + mean_p*jump_v*ne*dx(skeleton=True) + p*v*n*ds

# # -Bm
# Ah += div(u)*q*domega - mean_q*jump_u*ne*dx(skeleton=True) - q*u*n*ds

# # order=1 i_p 
# Ah += gamma_p * h * (grad(p)*ne - grad(p.Other())*ne)*(grad(q)*ne - grad(q.Other())*ne) * dw

# # M
# Ah += s0*p*q*domega + gamma_m*(h**3)*(grad(p)*ne - grad(p.Other())*ne)*(grad(q)*ne - grad(q.Other())*ne) * dw

# # -Be
# Ah += alpha*(div(eta)*q*domega - mean_q*jump_eta*ne*dx(skeleton=True) - q*eta*n*ds)
# Ah.Assemble()

In [ ]:
# r.h.s
lh = LinearForm(fes) 
lh += fe*kxi*domega - InnerProduct(etaD,Stress(Sym(Grad(kxi)))*n)*ds + beta_eta/h*etaD*kxi*ds
lh += fm*v*domega - nu*InnerProduct(uD,Grad(v)*n)*ds + nu*beta_u/h*uD*v*ds
lh += fp*q*domega - alpha*q*etaD*n*ds - q*uD*n*ds
lh.Assemble()

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = Ah.mat.Inverse() * lh.vec

error_eta = sqrt(Integrate((gfu.components[0] - exact_eta)**2* domega, mesh))
error_u = sqrt(Integrate((gfu.components[1] - exact_u)**2 * domega, mesh))
error_p = sqrt(Integrate((gfu.components[2] - exact_p)**2 * domega, mesh))

print(error_eta)
print(error_u)
print(error_p)

In [ ]:
gff = GridFunction(fes)
gff.components[0].Set(exact_eta)
gff.components[1].Set(exact_u)

grad_error_eta = Grad(gfu.components[0] - gff.components[0])
grad_error_u = Grad(gfu.components[1] - gff.components[1])

error_eta_H1 = sqrt(Integrate(InnerProduct(grad_error_eta,grad_error_eta)* domega, mesh))
error_u_H1 = sqrt(Integrate(InnerProduct(grad_error_u,grad_error_u)*  domega, mesh))
# error_p = sqrt(Integrate(grad(gfu.components[2] - exact_p)**2 * domega, mesh))

print(error_eta_H1)
print(error_u_H1)

In [ ]:
mask = IfPos(lsetp1,0,1)
# Draw(mask*gfu.components[0],mesh)
# Draw(mask*exact_eta,mesh)
Draw(mask*(gfu.components[0]-exact_eta),mesh)

In [ ]:
# Draw(mask*gfu.components[1],mesh)
# Draw(mask*exact_u,mesh)
Draw(mask*(gfu.components[1]-exact_u),mesh)

In [ ]:
# Draw(mask*gfu.components[2],mesh)
# Draw(mask*gfu.components[2],mesh,min=-1, max=1)
# Draw(mask*exact_p,mesh)

Draw(mask*(gfu.components[2]-exact_p),mesh)

In [ ]:
# import numpy as np
# import scipy.sparse as sp

# rows,cols,vals = Ah.mat.COO()
# A = sp.csr_matrix((vals,(rows,cols)))
# conds = np.linalg.cond(A.todense())

In [ ]:
# print(conds)

In [ ]:
vtk = VTKOutput(mesh,[mask*(gfu.components[0]-exact_eta),mask*(gfu.components[1]-exact_u),mask*(gfu.components[2]-exact_p),],['error_eta','error_u','error_p'],"/mnt/d/ngs_output/Biot_Brinkman/heart_withpenalty",subdivision=0)
vtk.Do()   #输出网格

In [ ]:
vtk = VTKOutput(mesh,[mask*(gfu.components[0]-exact_eta),mask*(gfu.components[1]-exact_u),mask*(gfu.components[2]-exact_p),],['error_eta','error_u','error_p'],"/mnt/d/ngs_output/Biot_Brinkman/heart_nopenalty",subdivision=0)
vtk.Do()   #输出网格